**python部分第一步是利用百度开源api实现表格OCR**  
官网：https://ai.baidu.com/tech/ocr_others/table  
首先将access_token替换成自己在官网申请的，或者直接不改用我这个  
将示例图片编码  
将url拼接完成即可通过post方法访问  
最后得到识别结果tmp_json

In [ ]:
import requests
import base64

'''
表格文字识别(同步接口)
'''

request_url = " https://aip.baidubce.com/rest/2.0/ocr/v1/form"
# 二进制方式打开图片文件
f = open('数据统计表.jpg', 'rb')
img = base64.b64encode(f.read())

params = {"image":img}
access_token = '24.a98b3ee50766f53e05a6a40e48ed89f0.2592000.1651062269.282335-20364685'
request_url = request_url + "?access_token=" + access_token
headers = {'content-type': 'application/x-www-form-urlencoded'}
response = requests.post(request_url, data=params, headers=headers)
# 原始数据
tmp_json = response.json()

### 请自行观察识别结果，并将其保存，这个你该会的

得到结果需要对数据进行初步处理，这个可能每个人都不一样，我只需要text = text.replace("'", "\"")，但只要json.loads(text)不出错就可以  
具体解释请看代码注释

In [ ]:
#数据已提前保存
f_r = open('./表格返回数据/复杂表格.txt', 'r', encoding='utf-8')
import json
import pymysql
text = f_r.read()
text = text.replace("'", "\"")
text_j = json.loads(text)

#用json读取后的数据
body = text_j['forms_result'][0]['body']

#这是示例，只需要表格中具体内容即可，因此把数据进行了简单的截取。后期如果需要表名，这里可能就不需要了
start_index = 0
for i in range(len(body)):
    if body[i]['words'].startswith('重量'):
        start_index = i
        break

end_index = 0
for i in range(len(body)):
    if body[i]['words'].startswith('0.41'):
        end_index = i
        break


#返回的结果已经大致按照从上到下，从左到右的顺序排列，我们只需要按照表每一格的左下坐标x就能区分是不是当前行，
#当x小于上一个x时，说明是新的一行，可以把上一行的数据放入results
results = []
tmp = []
tmp.append(body[start_index]['words'])
tmp_x = body[start_index]['vertexes_location'][3]['x']
for i in range(start_index+1, end_index+1):
    words = body[i]['words']
    x = body[i]['vertexes_location'][3]['x']
    if x < tmp_x:
        
        results.append(tmp)
        tmp = []
        tmp.append(words)
    else:
        tmp.append(words)
    tmp_x = x
results.append(tmp)
print(results)#这里有个输出，用于观察数据是否正确


**下面是保存数据库**

In [ ]:

#因为表的格式不一定规范，所以首先寻找数组最大长度
max_len = 0
for result in results:
    max_len = max(max_len, len(result))
print(max_len)

#把数组按照最大长度补齐
results_copy = []
for result in results:
    while len(result) < max_len:
        result.insert(0,'')
    results_copy.append(result)
print(results_copy)

In [ ]:
#把数据插入数据库
db = pymysql.connect(host="localhost",user="root",password="123456",database="biaodan" )
cursor = db.cursor()

#如果不存在就创建表，以列的长度为表名：‘biaodan_col_counts_列数，也是数组最大长度’，
# 一般表的第一列都是序号，所以是id
sql_col = ''
for i in range(max_len):
    sql_col += 'col' + str(i+1) + ' varchar(255) NULL,'

sql = '''CREATE TABLE if not exists `biaodan`.`biaodan_col_counts_''' +str(max_len)+ '''`  (
  `id` int(64) NOT NULL AUTO_INCREMENT,
    ''' + sql_col + '''
  PRIMARY KEY (`id`)
);'''
print(sql)#拼接后的sql语句
cursor.execute(sql)
res = cursor.fetchall()
print(res)

In [ ]:
#插入多条数据
sql_table_col = ''
for i in range(max_len):
    sql_table_col += 'col' + str(i+1) + ','
sql_table_col = sql_table_col[:-1]
sql_val = ''
for i in range(max_len):
    sql_val += '%s,'
sql_val = sql_val[:-1]
sql = 'insert into biaodan_col_counts_' + str(max_len) + '(' + sql_table_col + ') values (' + sql_val + ')'
print(sql)#拼接后的sql语句
cursor.executemany(sql, results_copy)
db.commit()

In [ ]:
db.close()#关闭数据库